In [4]:
import sys
import os

sys.path.append(f'{os.getcwd()}/..')

import numpy as np
import pandas as pd
import seaborn as sn
from matplotlib import pyplot as plt

from sklearn.linear_model import LogisticRegressionCV

from collections import Counter

from src.class_computer import link_prediction_methods

In [43]:
df = pd.read_csv('../data/precomputed_with_classes_3.csv')
del df['class']
df = df.dropna()

In [11]:
in_cols = list(df.columns[3:-len(link_prediction_methods)])
out_cols = link_prediction_methods

In [60]:
from keras import Sequential, Input
from keras.layers import Dense

in_dim = len(in_cols)
out_dim = len(out_cols)

def build_nn_model():
    model = Sequential([
        Dense(in_dim*2, input_dim=in_dim, activation='relu'),
        Dense(in_dim*2, activation='relu'),
        Dense(out_dim),
    ])
    model.compile(optimizer='adam', loss='mse', metrics=['accuracy', 'mae'])
    return model


In [67]:
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.preprocessing import Normalizer
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df[in_cols], df[out_cols], test_size=0.10, random_state=0)

normalizer = Normalizer().fit(X_train)

X_train = normalizer.transform(X_train)
Y_train = y_train

X_test = normalizer.transform(X_test)
Y_test = y_test

regressor = KerasRegressor(build_fn=build_nn_model, batch_size=32,epochs=100)
regressor.fit(X_train, Y_train)

Epoch 1/100
1477/1477 [==============================] - 1s 375us/step - loss: 0.2928 - accuracy: 0.2925 - mae: 0.4996
Epoch 2/100
1477/1477 [==============================] - 0s 176us/step - loss: 0.0408 - accuracy: 0.2918 - mae: 0.1564
Epoch 3/100
1477/1477 [==============================] - 0s 249us/step - loss: 0.0149 - accuracy: 0.2844 - mae: 0.0811
Epoch 4/100
1477/1477 [==============================] - 0s 238us/step - loss: 0.0134 - accuracy: 0.3108 - mae: 0.0762
Epoch 5/100
1477/1477 [==============================] - 0s 181us/step - loss: 0.0121 - accuracy: 0.3284 - mae: 0.0719
Epoch 6/100
1477/1477 [==============================] - 0s 206us/step - loss: 0.0107 - accuracy: 0.3575 - mae: 0.0670
Epoch 7/100
1477/1477 [==============================] - 0s 225us/step - loss: 0.0095 - accuracy: 0.3615 - mae: 0.0621
Epoch 8/100
1477/1477 [==============================] - 0s 198us/step - loss: 0.0084 - accuracy: 0.3636 - mae: 0.0584
Epoch 9/100
1477/1477 [=========================

In [74]:
Y_test.iloc[0:10, :]

,resource_allocation,jaccard_coefficient,preferential_attachment,community,sorensen_neighbours
1575,0.839080,0.919540,0.574713,0.666667,0.919540
1420,0.858289,0.834225,0.334225,0.713904,0.839572
991,0.951945,0.955378,0.411899,0.866133,0.947368
1399,0.931677,0.936335,0.338509,0.784161,0.936335
1579,0.898585,0.825472,0.365566,0.702830,0.865566
596,0.968354,0.965190,0.477848,0.778481,0.977848
1124,0.949640,0.944844,0.405276,0.856115,0.952038
1564,0.917460,0.930159,0.377778,0.860317,0.933333
1333,0.836864,0.879237,0.411017,0.760593,0.868644
1512,0.944444,0.942460,0.404762,0.857143,0.958333


In [97]:
results = regressor.predict(X_test)
results[0:20, :]

array([[0.81846166, 0.7835689 , 0.5038661 , 0.6651687 , 0.7692672 ],
       [0.8767184 , 0.8997793 , 0.3643569 , 0.79295105, 0.8936988 ],
       [0.9588994 , 0.97369677, 0.3773047 , 0.8477847 , 0.97288746],
       [0.92923963, 0.94758785, 0.37145168, 0.82930124, 0.9445426 ],
       [0.86901855, 0.8969667 , 0.34711322, 0.79245275, 0.8896325 ],
       [0.9558034 , 0.97104335, 0.3669403 , 0.83798766, 0.9708549 ],
       [0.9445344 , 0.960672  , 0.37928718, 0.83835393, 0.95855635],
       [0.9314136 , 0.9496528 , 0.36111304, 0.82579774, 0.94777936],
       [0.8681029 , 0.88922703, 0.36896247, 0.7881229 , 0.8835571 ],
       [0.9450444 , 0.9628157 , 0.36804304, 0.83713883, 0.96087664],
       [0.62362504, 0.6334057 , 0.3127833 , 0.7549271 , 0.63862056],
       [0.8460606 , 0.86499244, 0.35732445, 0.7519562 , 0.8568853 ],
       [0.9002975 , 0.92026687, 0.3491668 , 0.7930862 , 0.91489774],
       [0.9100089 , 0.92883646, 0.38775292, 0.8138307 , 0.9236721 ],
       [0.91436684, 0.93568957, 0.

In [99]:
predicted = list(map(lambda x: link_prediction_methods[x], np.argmax(results, axis=1)))
predicted[0:20]

['resource_allocation',
 'jaccard_coefficient',
 'jaccard_coefficient',
 'jaccard_coefficient',
 'jaccard_coefficient',
 'jaccard_coefficient',
 'jaccard_coefficient',
 'jaccard_coefficient',
 'jaccard_coefficient',
 'jaccard_coefficient',
 'community',
 'jaccard_coefficient',
 'jaccard_coefficient',
 'jaccard_coefficient',
 'jaccard_coefficient',
 'community',
 'jaccard_coefficient',
 'jaccard_coefficient',
 'jaccard_coefficient',
 'resource_allocation']

In [175]:
import statistics

Y_score = Y_test.reset_index(drop=True)

aucs = []
for i, p in enumerate(predicted):
    aucs.append(Y_score.loc[i, p])

(statistics.mean(aucs), statistics.stdev(aucs))

(0.8667750401235926, 0.1124510633993589)

#### Baseline
- AUC mean: 0.859299
- AUC std: 0.132959

#### NN Regression
- AUC mean: 0.8667750401235926
- AUC std: 0.1124510633993589